In [51]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
 
import random
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import cv2
import PIL
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [18]:
def conv_block(input, num_filters):
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = layers.concatenate([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = layers.Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = conv_block(p4, 1024)

    # Decoder
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Output
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs)
    return model

input_shape = (112, 112, 3)  # Adjust size and channels according to your dataset
model = build_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [77]:

# Paths to your data.
images_path = './archive/crack_segmentation_dataset/train/images/'
masks_path = './archive/crack_segmentation_dataset/train/masks/'
img_size = (112, 112)

# Load and preprocess data
def load_data(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    files = []
    count = 0
    for filename in os.listdir(images_path):
        if count > 500:
            break
        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename
        # print(img_path)
        # Load and preprocess the image
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        # print(img.size())
    #   if not img or not mask:
    #         continue
        img, mask = img/255.0, mask/255.0
        # img = cv2.resize(img, img_size) / 255.0
        # # Load and preprocess the mask using OpenCV
        # mask = cv2.resize(mask, img_size) / 255.0
 
        for i in range(4):
            for j in range(4):
                img_part = img[i * 112: (i + 1) * 112,
                            j * 112: (j + 1) * 112]
                mask_part = mask[i * 112: (i + 1) * 112,
                            j * 112: (j + 1) * 112]
                images.append(img_part)
                masks.append(mask_part)
        files.append(filename)
         
        count += 1
    print(files)
    return np.array(images), np.array(masks)


In [76]:
def load_test(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    count = 0
    print()
    imgList = os.listdir(images_path)
    files = []
    for i in range(len(imgList) - 1, -1, -1):
        filename = imgList[i]
    # for filename in os.listdir(images_path):
        if count > 100:
            break

        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename

        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        img, mask = img/255.0, mask/255.0

        i, j = random.randint(0, 3), random.randint(0, 3)
        img_part = img[i * 112: (i + 1) * 112,
                    j * 112: (j + 1) * 112]
        mask_part = mask[i * 112: (i + 1) * 112,
                    j * 112: (j + 1) * 112]
        images.append(img_part)
        masks.append(mask_part)

        files.append(filename)
        count += 1
    print(files)
    return np.array(images), np.array(masks)

In [20]:
# Load your data
images, masks = load_data(images_path, masks_path, (112, 112))


In [21]:
images

array([[[[0.5254902 , 0.5372549 , 0.52941176],
         [0.50588235, 0.51764706, 0.50980392],
         [0.48235294, 0.49411765, 0.48627451],
         ...,
         [0.5372549 , 0.52941176, 0.52941176],
         [0.53333333, 0.51372549, 0.51764706],
         [0.51764706, 0.49803922, 0.50196078]],

        [[0.49411765, 0.50588235, 0.49803922],
         [0.48235294, 0.49411765, 0.48627451],
         [0.48235294, 0.49411765, 0.48627451],
         ...,
         [0.5372549 , 0.52941176, 0.52941176],
         [0.55294118, 0.53333333, 0.5372549 ],
         [0.56078431, 0.54117647, 0.54509804]],

        [[0.46666667, 0.47843137, 0.47058824],
         [0.4627451 , 0.4745098 , 0.46666667],
         [0.47058824, 0.48235294, 0.4745098 ],
         ...,
         [0.53333333, 0.5254902 , 0.5254902 ],
         [0.54901961, 0.52941176, 0.53333333],
         [0.56470588, 0.54509804, 0.54901961]],

        ...,

        [[0.50588235, 0.49803922, 0.49411765],
         [0.51764706, 0.50980392, 0.50588235]

In [22]:
images.shape

(8016, 112, 112, 3)

In [72]:
images1, masks1 = images[:500], masks[:500]

In [73]:
masks1 = np.expand_dims(masks1, axis=-1)
# Load your data
imgval, maskval = load_test(images_path, masks_path, (112, 112))

maskval = np.expand_dims(maskval, axis=-1)

In [25]:
masks1.shape

(500, 112, 112, 1)

In [74]:

X_train, y_train, X_val, y_val = images1, masks1,imgval, maskval 
# y_train.shape
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (500, 112, 112, 3)
y_train shape: (500, 112, 112, 1)
X_val shape: (101, 112, 112, 3)
y_val shape: (101, 112, 112, 1)


In [75]:


# Create data generators with desired augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()  # No augmentation for validation data

# Generate batches of augmented data from arrays (X_train and y_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

# Train the model using the fit_generator function
history = model.fit(
    train_generator, 
    steps_per_epoch=len(X_train) // 32,  # Number of batches per epoch
    epochs=10, 
    validation_data=val_generator,
    validation_steps=len(X_val) // 32  # Number of batches for validation
)


Epoch 1/10


c:\Users\link1\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.9695 - loss: 0.0836 - val_accuracy: 0.9436 - val_loss: 0.1990
Epoch 2/10
 1/15 ━━━━━━━━━━━━━━━━━━━━ 49s 4s/step - accuracy: 0.9774 - loss: 0.0699

c:\Users\link1\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9774 - loss: 0.0373 - val_accuracy: 0.9360 - val_loss: 0.1088
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.9691 - loss: 0.0833 - val_accuracy: 0.9403 - val_loss: 0.2414
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9677 - loss: 0.0451 - val_accuracy: 1.0000 - val_loss: 0.0317
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.9677 - loss: 0.0850 - val_accuracy: 0.9440 - val_loss: 0.2630
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9585 - loss: 0.0503 - val_accuracy: 0.9299 - val_loss: 0.1081
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.9701 - loss: 0.0789 - val_accuracy: 0.9438 - val_loss: 0.2148
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9596 - loss: 0.0506 - val_accuracy: 0.9331 - val_loss: 0.1163
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.9705 - loss: 0.0779 - val_accuracy: 0.9463 - val_loss: 0.2907

In [80]:

# Load and preprocess the image
image_path = 'CFD_001.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (112, 112))  # Resize to match your model's input shape
# image = image / 255.0  # Normalize the pixel values
cv2.imwrite( "asg.jpg", image)
# Convert the image to a NumPy array with the expected shape
# input_image = np.expand_dims(image, axis=0)  # Add a batch dimension
prediction = model.predict(input_image)
prediction

NameError: name 'input_image' is not defined

In [79]:
import numpy as np
import matplotlib.pyplot as plt

prediction *= 255

prediction_2d = prediction.squeeze()

plt.figure(figsize=(8, 8))
plt.imshow(prediction_2d, cmap='gray')
plt.colorbar()
plt.axis('off')
plt.show()

NameError: name 'prediction' is not defined